# NarniaLaudry

## Intro
Our client NarniaLaundry is looking to expand to Toronto by acquiring existing laundry services. 
Their experts have already identified several factors that guarantee viability of a their proprietary recipe of FSLS(Fast Sport Laundry Service) in other countries. They provide sports enthusiasts with the ability to wash their equipment in 15 minute and also store it until the next use.
They are now looking for possible emplacements  to reduce the risk of failure of a given laundry station. 
Previous experience has shown a correlation between successful stations and the density of gyms and coffee shops. 

Therefore our customer is looking for locations within walking distamce from:
* places where people sweat a lot and they come with gym bags.
  * yoga studios 
  * gym studios
  * dojos 
  * martial arts studios
* coffee shops
  * stand up coffee shops
  * take away coffee
  * not coffeeshops where they serve more than some light patisery
  * not restaurants
  * not automatic coffee dispensers
  

In [14]:

!conda install -c conda-forge pandas --yes 
!conda install -c conda-forge geocoder --yes 
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge tqdm --yes
!conda install -c conda-forge folium=0.5.0 --yes

# import itertools
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.ticker import NullFormatter
# import pandas as pd
# import numpy as np
# import matplotlib.ticker as ticker
# from sklearn import preprocessing
# %matplotlib inline


import numpy as np
import pandas as pd


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    pandas: 0.21.0-py35hee8c687_1 --> 0.23.4-py35hf8a1672_0 conda-forge

pandas-0.23.4- 100% |################################| Time: 0:00:00  90.22 MB/s
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geocoder                  1.38.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.19.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at 

## The dataset
Our dataset is a list of all the venues in Torronto that we are going to analyze.

We scrape Wikipedia for a up to date list of Neighbourhood and then we retrieves the coordinates,
Having a list of neighbourhoods with their coordinates, we are going to extract venues using data from Foursquare. 

It is we this data we are going to and determine the neighbourhoods that are most likely to support our client's business.

### Scrape the Wikipedia page

In [66]:
wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
wiki[0].head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


### Create the frame

In [75]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighbourhood
df = pd.DataFrame(wiki[0])
df.rename(columns=df.iloc[0])
df = df.drop(df.index[0]) 
df.rename(index=str, columns={0: 'PostalCode',
                             1: 'Borough',
                             2: 'Neighbourhood'},
          inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


### Clean the frame

In [86]:
df.replace({'Borough': 'Not assigned' }, 
           np.nan,
           inplace = True)
df.dropna(subset=["Borough"],
          inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [87]:
print(df.shape)

(103, 3)


#### If not assigned neighbourhood, then the neighbourhood will be the same as the borough

In [88]:
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned',
                               df['Borough'], 
                               df['Neighbourhood'])
assert(df.loc[df['Neighbourhood'] == 'Not assigned'].shape == (0,3))

#### Merge neighbourhoods in one postal code area.

In [89]:
df = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

### Adding Latitude and Longitude

In [90]:
gf = pd.read_csv('https://cocl.us/Geospatial_data')
gf.rename(index=str, columns={'Postal Code': 'PostalCode'}, inplace=True)
df = pd.merge(df, gf, how='inner', on=['PostalCode'])
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Showing the area we are interested in

In [92]:
latitude = df["Latitude"].mean()
longitude = df["Longitude"].mean()

import folium
tmap = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
folium.CircleMarker(
    [latitude, longitude],
    radius=5,
    popup='Toronto',
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.6,
    parse_html=False).add_to(tmap) 

  # add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tmap)
    
tmap

### Enhancing the data with info about other venues

In [96]:
CLIENT_ID = '4UDQN1GGFP32A2SVSR1LZ5KXYRQO2YZ3FZPVF3FOO03YGVRO' # your Foursquare ID
CLIENT_SECRET = '41D3Z4Z5LTGUBZR3XHMIX0YWCMN3WPBVCDWBXYNCM3D1DOYP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
    
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

venues = getNearbyVenues(names=df['Neighbourhood'],
                         latitudes=df['Latitude'],
                         longitudes=df['Longitude']
                        )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [100]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 275 uniques categories.


In [102]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
onehot['Neighbourhood'] = venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
grouped = onehot.groupby('Neighbourhood').mean().reset_index()

In [109]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] =grouped['Neighbourhood']

for ind in np.arange(grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Bar,Thai Restaurant,Café,Steakhouse,American Restaurant,Restaurant,Burger Joint,Sushi Restaurant,Gym
1,Agincourt,Lounge,Sandwich Place,Clothing Store,Breakfast Spot,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Beer Store,Fried Chicken Joint,Japanese Restaurant,Fast Food Restaurant,Liquor Store,Discount Store,Pharmacy,Pizza Place,Sandwich Place
4,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Pharmacy,Pool,Pub,Dance Studio,Sandwich Place,Coffee Shop,General Entertainment
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Sandwich Place,Supermarket,Restaurant,Bank,Deli / Bodega,Fried Chicken Joint,Frozen Yogurt Shop,Fast Food Restaurant,Sushi Restaurant
6,Bayview Village,Chinese Restaurant,Café,Japanese Restaurant,Bank,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
7,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Pharmacy,Café,Restaurant,Indian Restaurant,Sushi Restaurant,Pub,Juice Bar
8,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Café,Seafood Restaurant,Restaurant,Pub,Cheese Shop,Steakhouse,Farmers Market
9,"Birch Cliff, Cliffside West",General Entertainment,College Stadium,Café,Skating Rink,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
